In [129]:
import os
import pathlib
from pathlib import Path 
import pandas as pd
import numpy as np
import re

To do:
* Remove duplicate rows
* Check for different text referring to same facility eg 'hospital' vs 'chp'

In [181]:
df_raw = pd.read_excel('Health Facility Assessment (1).xlsx',index_col=0, usecols='B:GW')
df = df_raw.copy()

Let's start by displaying the column names:

In [182]:
cols = list(df_raw.columns)
print(*cols,sep='\n')

form.health_centre_information.facility_name
form.facility_gps
form.health_centre_information.location_information.region_province
form.health_centre_information.location_information.district
form.health_centre_information.location_information.chiefdom
form.health_centre_information.location_information.facility_location
form.health_centre_information.facility_type
form.health_centre_information.facility_type_other
form.health_centre_information.managing_authority
form.health_centre_information.managing_authority_other
form.health_centre_information.setting
form.health_centre_information.outpatient_only
form.health_centre_information.capacity.number_consultation_rooms
form.health_centre_information.capacity.number_inpatient_beds
form.health_centre_information.capacity.number_maternity_beds
form.health_centre_information.group_number_employed.list_community_health_officer.cho_number_in_post
form.health_centre_information.group_number_employed.list_community_health_officer.cho_number_pre

These column names are painfully long to work with, but I don't want to make them less informative, so I will settle for making them a bit less shorter by removing a few unnecessary words.

In [183]:
new_cols = []
for i,name in enumerate(cols):
    name = re.sub('form\.','',name)
    name = re.sub('health_centre_information\.','',name)
    name = re.sub('location_information\.','',name)
    new_cols.append(name)

    print(name)
df.columns = new_cols

facility_name
facility_gps
region_province
district
chiefdom
facility_location
facility_type
facility_type_other
managing_authority
managing_authority_other
setting
outpatient_only
capacity.number_consultation_rooms
capacity.number_inpatient_beds
capacity.number_maternity_beds
group_number_employed.list_community_health_officer.cho_number_in_post
group_number_employed.list_community_health_officer.cho_number_present_at_visit
group_number_employed.list_community_health_technician.cht_number_in_post
group_number_employed.list_community_health_technician.cht_number_present_at_visit
group_number_employed.list_community_health_assistant.cha_number_in_post
group_number_employed.list_community_health_assistant.cha_number_present_at_visit
group_number_employed.list_community_midwives.cm_number_in_post
group_number_employed.list_community_midwives.cm_number_present_at_visit
group_number_employed.list_sechn.sechn_number_in_post
group_number_employed.list_sechn.sechn_number_present_at_visit
group

In [184]:


# split GPS values into longitude and latitude
df.insert(loc=1,column = 'facility_gps_lat',value=df['facility_gps'].apply(lambda x: x.split(',')[0]))
df.insert(loc=2,column = 'facility_gps_lon',value=df['facility_gps'].apply(lambda x: x.split(',')[1]))
#df = df.drop('facility_gps',axis=1)

# replace --- values with NA
df = df.replace('---', np.nan, regex=True)

# make location names all uppercase so that differences in capitalisation don't prevent matches
df['facility_name'] = df['facility_name'].str.upper()
df['region_province'] = df['region_province'].str.upper()
df['district'] = df['district'].str.upper()
df['chiefdom'] = df['chiefdom'].str.upper()
df['facility_location'] = df['facility_location'].str.upper()

In [185]:
df

,facility_name,facility_gps_lat,facility_gps_lon,facility_gps,region_province,district,chiefdom,facility_location,facility_type,facility_type_other,...,grp_infection_prevention_and_control.ql_water_sanitation_and_hygiene.score_infection_prevention_and_control_water_sanitation_and_hygiene,grp_infection_prevention_and_control.ql_water_sanitation_and_hygiene.score_max_infection_prevention_and_control_water_sanitation_and_hygiene,grp_infection_prevention_and_control.ql_disinfection_and_sterilization.score_infection_prevention_and_control_disinfection_and_sterilization,grp_infection_prevention_and_control.ql_disinfection_and_sterilization.score_max_infection_prevention_and_control_disinfection_and_sterilization,grp_infection_prevention_and_control.score_infection_prevention_and_control,grp_infection_prevention_and_control.score_max_infection_prevention_and_control,question1.score_logistics_patient_and_sample_transfer,question1.score_max_logistics_patient_and_sample_transfer,score_total,score_max_total
number,,,,,,,,,,,,,,,,,,,,,
0,WILLIAM HOSPITAL,60.234598,-90.157892,"60.234598, -90.157892",SOUTHERN,PUJEHUN,NEW YORK,JASONFORT,community_health_post,NaN,...,4,5,1,5,10.5,27,19.5,44,48.5,102.0
1,KATHRYN HOSPITAL/CHC,21.2454855,-107.509120,"21.2454855, -107.509120",SOUTHERN,BO,ARIZONA,WEST JAMESSHIRE,community_health_post,NaN,...,1.5,5,0,5,5.5,27,21,44,35.5,102.0
2,JESSICA HOSPITAL,-0.178657,-9.190029,"-0.178657, -9.190029",SOUTHERN,BO,NEW MEXICO,ROBERTSSIDE,community_health_post,NaN,...,2,5,0,5,10,27,33.5,44,55.5,102.0
3,LAUREN CHC,35.856989,-87.512790,"35.856989, -87.512790",SOUTHERN,BONTHE,MISSOURI,PORT CHRISTOPHER,mch_post,NaN,...,4,5,2,5,12,27,34.5,44,57.5,102.0
4,TAMMY CHC,54.7148835,150.744813,"54.7148835, 150.744813",SOUTHERN,BONTHE,VIRGINIA,RENEETON,community_health_post,NaN,...,4,5,3,5,12,27,29.5,44,55.0,102.0
5,NaN,22.8567535,19.262410,"22.8567535, 19.262410",NaN,NaN,NaN,NaN,NaN,NaN,...,0,5,0,5,0,27,NaN,NaN,NaN,NaN
6,BRYAN CLINIC,61.8040035,28.191714,"61.8040035, 28.191714",SOUTHERN,PUJEHUN,MICHIGAN,ADAMMOUTH,community_health_post,NaN,...,1,5,2,5,8.5,27,21,44,53.0,102.0
7,CHERYL HOSPITAL/CHC,86.9708915,173.527045,"86.9708915, 173.527045",SOUTHERN,BONTHE,NEVADA,BRYANCHESTER,community_health_centre,NaN,...,3,5,1,5,7,27,15,44,38.0,102.0
8,BRANDI HOSPITAL,77.8973365,70.112618,"77.8973365, 70.112618",SOUTHERN,MOYAMBA,FLORIDA,PORT KRISTINVILLE,community_health_centre,NaN,...,2.5,5,4,5,19.5,27,25,44,65.5,102.0


Let's investigate duplicate facility names:

In [187]:
print(df[df.duplicated(['facility_name','facility_location'],keep=False)].sort_values('facility_location'))

                   facility_name facility_gps_lat facility_gps_lon  \
number                                                               
47                 JOANNA CLINIC       60.1909345       162.787117   
25                 JOANNA CLINIC       15.7540145      -153.871776   
86                MELISSA CLINIC        35.576733       179.184687   
113               MELISSA CLINIC      -29.0475445      -115.874086   
96                  RICHARD MCHP      -26.3419165      -172.661039   
97                  RICHARD MCHP      -81.3326275        69.862048   
48                       AMY CHP        84.498725        48.271191   
87                       AMY CHP        70.462361       122.083912   
95                   JOHN CLINIC        53.647814      -115.559792   
26                   JOHN CLINIC      -54.3882585      -131.023446   
39                KEVIN HOSPITAL       -53.597735       136.778861   
133               KEVIN HOSPITAL        41.570610        66.484104   
140     NICOLE TERTI

Strange - there are a number of duplicate facility names with very different GPS coordinates. 

In [92]:
df.dtypes

form.health_centre_information.facility_name                                                                                                              object
form.facility_gps_lon                                                                                                                                     object
form.facility_gps_lat                                                                                                                                     object
form.health_centre_information.location_information.region_province                                                                                       object
form.health_centre_information.location_information.district                                                                                              object
form.health_centre_information.location_information.chiefdom                                                                                              object
form.health_centre_information.loc

Unnamed: 0
number
form.health_centre_information.facility_name
form.facility_gps
form.health_centre_information.location_information.region_province
form.health_centre_information.location_information.district
form.health_centre_information.location_information.chiefdom
form.health_centre_information.location_information.facility_location
form.health_centre_information.facility_type
form.health_centre_information.facility_type_other
form.health_centre_information.managing_authority
form.health_centre_information.managing_authority_other
form.health_centre_information.setting
form.health_centre_information.outpatient_only
form.health_centre_information.capacity.number_consultation_rooms
form.health_centre_information.capacity.number_inpatient_beds
form.health_centre_information.capacity.number_maternity_beds
form.health_centre_information.group_number_employed.list_community_health_officer.cho_number_in_post
form.health_centre_information.group_number_employed.list_community_health_offi

In [30]:
print(df['form.health_centre_information.location_information.facility_location'].unique())

['Jasonfort' 'West Jamesshire' 'Robertsside' 'Port Christopher' 'Reneeton'
 nan 'Adammouth' 'Bryanchester' 'Port Kristinville' 'South Adam'
 'Harrisburgh' 'Martinbury' 'Brownland' 'West Alexisport' 'Perezside'
 'Townsendmouth' 'Valerieborough' 'East Brandon' 'Colleenburgh'
 'New Heiditown' 'East Beverlymouth' 'Haileyshire' 'Markview'
 'Amberchester' 'Garytown' 'Lisahaven' 'Lake Sean' 'Lake Jennifer'
 'North Sara' 'Kerrville' 'Lake Johnstad' 'Port Colleenfort'
 'West Jennifer' 'Riceville' 'Donnaburgh' 'Hunterport' 'Kaiserberg'
 'Port Carolburgh' 'Port Genehaven' 'New Walterport' 'Melissaport'
 'Collinston' 'East Jamesberg' 'North Maria' 'Johnsonfurt' 'Mitchellport'
 'South Amy' 'Craigside' 'Sheppardhaven' 'North Ronaldfort' 'Philipland'
 'Gibsonstad' 'Brooketon' 'Smithmouth' 'Hawkinsfurt' 'Brandonport'
 'Wallacemouth' 'North Amber' 'Robertburgh' 'New Cynthia' 'Ashleemouth'
 'New Karen' 'New Ashley' 'Paulfort' 'New Craig' 'Kristyview'
 'West Peterberg' 'Ariaston' 'Ericaburgh' 'Murrayport

In [93]:
df.to_csv('health_facility_assessment_cleaned.csv', header=True, index=True, index_label=None)